In [ ]:
# The idea of this notebook is to have a look at what is actually going on with the states and if predictions make sense

In [3]:
import pandas as pd
from chess_ai.rlagent.muzero.utils import MOVES, encode_state
from chess_ai.rlagent.muzero.models import AlphazeroNetSupervised
from chess_ai.rlagent.muzero.utils import get_root_dir
import torch
import numpy as np
from chess_python.chess import Chess
from pathlib import Path

In [15]:
import os
os.getcwd()

'/Users/pablocanadapereira/Desktop/Projects/chess-ai/notebooks'

In [17]:
buffer = pd.DataFrame()
for i in range(4):
    buffer_ = pd.read_feather(Path("../") /"chess_ai"/ "rlagent" / "data" / "dataframe" / f"buffer_{i+1}_df.feather")
    buffer = pd.concat([buffer, buffer_])

In [19]:
df = buffer

In [2]:
#df = pd.read_feather("../supervised_dataset_2.feather")

In [36]:
#df["value"] = 1/(1+np.exp(-0.01*df.evaluation))

In [22]:
df.value.min()

2.2101565448298504e-36

In [20]:
df.iloc[1].state, df.iloc[1].value

(array([ 4,  0,  3,  5,  6,  3,  2,  4,  1,  1,  1,  1,  1,  1,  1,  1,  0,
         0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1, -1, -1,
        -1, -1, -1, -1, -1, -4, -2, -3, -5, -6, -3, -2, -4, -1,  0, 15]),
 0.5274723043445937)

In [53]:
pol = df.iloc[1].policy

In [54]:
MOVES[df.iloc[1].policy.argmax()]

'e7e5'

In [26]:
model = AlphazeroNetSupervised()
model.load_state_dict(torch.load("../chess_ai/rlagent/checkpoints/nn_supervised.pth"))
model.eval()

AlphazeroNetSupervised(
  (l1): Linear(in_features=67, out_features=100, bias=True)
  (l2): Linear(in_features=100, out_features=1, bias=True)
  (l3): Linear(in_features=100, out_features=4208, bias=True)
  (activation): Tanh()
)

In [47]:
torch.tensor(df.iloc[1].state).view(-1,67)

tensor([[ 4,  2,  3,  5,  6,  3,  2,  4,  1,  1,  1,  1,  0,  1,  1,  1,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1, -1, -1, -1, -1, -1,
         -1, -1, -4, -2, -3, -5, -6, -3, -2, -4, -1,  0, 15]])

In [46]:
print(encode_state(Chess().move("e2e4").state))

[4, 2, 3, 5, 6, 3, 2, 4, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, -4, -2, -3, -5, -6, -3, -2, -4, -1, 5, 15]


In [59]:
value_pred_train, policy_pred_train = model(torch.tensor(df.iloc[1].state, dtype=torch.float32).view(-1,67))
value_pred, policy_pred = model(torch.tensor(encode_state(Chess().move("e2e4").state), dtype=torch.float32).view(-1, 67))

In [60]:
print(value_pred_train, policy_pred_train)
print("Net pred", value_pred, policy_pred)

tensor([[0.5420]], grad_fn=<TanhBackward0>) tensor([[7.3826e-18, 7.3826e-18, 7.3826e-18,  ..., 7.3826e-18, 7.3826e-18,
         7.3826e-18]], grad_fn=<SoftmaxBackward0>)
Net pred tensor([[0.5926]], grad_fn=<TanhBackward0>) tensor([[5.1872e-18, 5.1872e-18, 5.1872e-18,  ..., 5.1872e-18, 5.1872e-18,
         5.1872e-18]], grad_fn=<SoftmaxBackward0>)


In [67]:
MOVES[policy_pred.argmax().detach().numpy()]

'e7e6'